In [2]:
%matplotlib inline

In [3]:
import matplotlib as mpl
mpl.use('Agg')

import pickle as pkl
import math
from graph_tool.draw import graph_draw
from matplotlib import  pyplot as plt

from graph_helpers import remove_filters, load_graph_by_name, gen_random_spanning_tree
from viz_helpers import lattice_node_pos, QueryIllustrator
from experiment import gen_input, one_round_experiment, remove_filters
from query_selection import RandomQueryGenerator, EntropyQueryGenerator, PRQueryGenerator, PredictionErrorQueryGenerator
from simulator import Simulator


/usr/local/lib/python3.5/dist-packages/matplotlib/__init__.py:1357: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


In [4]:
graph_name = 'lattice'
g = load_graph_by_name(graph_name)

In [5]:
if graph_name == 'lattice':
    pos = lattice_node_pos(g, shape=(10, 10))


In [13]:
use_cached_result = True

In [6]:
n_samples = 100
stop_fraction = 0.25
n_queries = 10

In [19]:
if not use_cached_result:
    obs, c = gen_input(g, stop_fraction=stop_fraction, p=0.5)
    # pkl.dump((obs, c), open('/tmp/cascade_example.pkl', 'wb'))
else:
    methods = ['random', 'pagerank', 'entropy', 'prediction_error']
    round_id = 84
    (obs, c) = pkl.load(open('cascade/{}/{}.pkl'.format(graph_name, round_id), 'rb'))
    queries_by_method = {
        method: pkl.load(open('outputs/queries/{}/{}/{}.pkl'.format(graph_name, method, round_id),
                              'rb'))[0]
        for method in methods}
    n_queries = len(queries_by_method['random'])

In [20]:
queries_by_method

{'entropy': [36, 24, 13, 25, 56, 46, 31, 30, 42, 74],
 'pagerank': [21, 65, 45, 34, 11, 56, 76, 32, 54, 43],
 'prediction_error': [48, 59, 37, 93, 5, 14, 1, 46, 91, 98],
 'random': [7, 68, 37, 53, 58, 46, 45, 14, 30, 81]}

In [8]:
n_plots = n_queries
n_row = 1
n_col = int(math.ceil(n_plots / n_row))

def create_fig_axes():
    plt.clf()
    fig, axes = plt.subplots(n_row, n_col, sharex=True, sharey=True)
    return fig, axes

In [22]:
stategies = [
    (PredictionErrorQueryGenerator, {'num_stt': n_samples}, 'prediction_error'), 
    (EntropyQueryGenerator, {'num_stt': n_samples, 'method': 'entropy', 'use_resample': False}, 'entropy'),
    (RandomQueryGenerator, {}, 'random'),
    (PRQueryGenerator, {}, 'pagerank'),
]


In [25]:
plt.switch_backend('cairo')

for cls, params, name in stategies:
    print(name)
    if not use_cached_result:
        gv = remove_filters(g)
        q_gen = cls(gv, **params)
        sim = Simulator(gv, q_gen, print_log=True)
        queries, _ = sim.run(n_queries, obs, c)
    else:
        # use the pre-computed queries
        queries = queries_by_method[name]

    qi = QueryIllustrator(g, obs, c, pos)
    fig, axes = create_fig_axes()
    for i, ax in enumerate(axes):
        qi.plot_snapshot(queries[i], n_samples=n_samples, ax=ax)
    fig.set_size_inches(5 * n_queries, 5, forward=True)
    fig.savefig('figs/query_process/{}.pdf'.format(name))

prediction_error
entropy
random
pagerank
